In [2]:
import numpy as np
import math
from dataclasses import dataclass, field
from typing import ClassVar, List, Optional, Dict
from __future__ import annotations
import pandas as pd

## Bus

In [2]:
@dataclass
class Bus:
    network: "Network"
    id: Optional[int] = None
    name: Optional[str] = None
    bus_type: str = "PQ"  # "slack", "PQ", "PV"
    v: float = 1.0 # in pu
    theta: float = 0.0 # in degrees

    # Relacionamentos
    loads: List["Load"] = field(default_factory=list)
    generators: List["Generator"] = field(default_factory=list)

    _id_counter: ClassVar[int] = 0

    def __post_init__(self):
        if self.id is None:
            self.id = Bus._id_counter
            Bus._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Bus._id_counter:
                Bus._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Bus {self.id}"

        # Add the bus to the network
        self.network.buses.append(self)
    
    @property
    def theta_rad(self) -> float:
        return np.deg2rad(self.theta)
    
    @property
    def p(self) -> float:
        """Net active power injection (pu)"""
        return sum(g.p for g in self.generators) - sum(l.p for l in self.loads)

    @property
    def q(self) -> float:
        """Net reactive power injection (pu)"""
        return sum(g.q for g in self.generators) - sum(l.q for l in self.loads) 
    
    # add_generator and add_load methods are used inside Generator and Load classes automatically. 
    # You just need to inform wich bus the generator or load is connected to.
    def add_generator(self, generator: 'Generator'):
        if generator not in self.generators:
            self.generators.append(generator)

    def add_load(self, load: 'Load'):
        if load not in self.loads:
            self.loads.append(load)

    def __repr__(self):
        return (f"Bus(id={self.id}, type={self.bus_type}, v={self.v:.3f} pu, "
                 f"theta={self.theta_rad:.3f} rad, p={self.p:.3f} pu, q={self.q:.3f} pu, "
                 f"gen={len(self.generators)}, load={len(self.loads)})")

## PowerDevice

In [3]:
@dataclass
class PowerDevice:
    bus: Bus
    name: Optional[str] = None
    pb: float = 1.0
    p_input: float = 0.0
    q_input: float = 0.0
    p_max_input: float = float('inf')
    p_min_input: float = 0.0
    q_max_input: Optional[float] = None
    q_min_input: Optional[float] = None
    cost_a_input: float = 0.0
    cost_b_input: float = 0.0
    cost_c_input: float = 0.0

    _counter: int = field(default=0, init=False, repr=False)

    @property
    def p(self) -> float:
        return self.p_input / self.pb
    
    @property
    def q(self) -> float:
        return self.q_input / self.pb

    @property
    def p_max(self) -> float:
        return self.p_max_input / self.pb
    
    @property
    def p_min(self) -> float:
        return self.p_min_input / self.pb
    
    @property
    def q_max(self) -> Optional[float]:
        return self.q_max_input / self.pb if self.q_max_input is not None else None

    @property
    def q_min(self) -> Optional[float]:
        return self.q_min_input / self.pb if self.q_min_input is not None else None

    @property
    def cost_a(self) -> float:
        return self.cost_a_input * self.pb
    
    @property
    def cost_b(self) -> float:
        return self.cost_b_input * self.pb
    
    @property
    def cost_c(self) -> float:
        return self.cost_c_input * self.pb

## Generator

In [4]:
@dataclass
class Generator(PowerDevice):
    id: Optional[int] = None
    _id_counter: ClassVar[int] = 0
    def __post_init__(self):
        if self.id is None:
            self.id = Generator._id_counter
            Generator._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Generator._id_counter:
                Generator._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Generator {self.id}"
            
        self.bus.add_generator(self)
        self.network = self.bus.network
        self.network.generators.append(self)

    def __repr__(self):
        return (f"Generator(id={self.id}, bus={self.bus.id}, p={self.p:.3f}, q={self.q:.3f}, "
                f"p_range=[{self.p_min:.3f},{self.p_max:.3f}], q_range=[{self.q_min},{self.q_max}])")

## Load

In [5]:
@dataclass
class Load(PowerDevice):
    id: Optional[int] = None

    _id_counter: ClassVar[int] = 0

    def __post_init__(self):
        if self.id is None:
            self.id = Load._id_counter
            Load._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Load._id_counter:
                Load._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Load {self.id}"
            
        self.bus.add_load(self)
        self.network = self.bus.network
        self.network.loads.append(self)

    def __repr__(self):
        return (f"Load(id={self.id}, bus={self.bus.id}, p={self.p:.3f}, q={self.q:.3f}, "
                f"p_range=[{self.p_min:.3f},{self.p_max:.3f}], q_range=[{self.q_min},{self.q_max}])")

## Line

In [6]:
@dataclass
class Line:
    from_bus: Bus
    to_bus: Bus
    id: Optional[int] = None
    name: Optional[str] = None
    pb: float = 1.0
    vb: float = 1.0
    r: float = 0.0
    x: float = 0.01
    b_half: float = 0.0
    flux_max: float = float('inf')
    tap_ratio: float = 1.0
    tap_phase: float = 0.0

    _id_counter: ClassVar[int] = 0  # ID counter for lines

    def __post_init__(self):
        # Set default values if not provided
        if self.id is None:
            self.id = Line._id_counter
            Line._id_counter += 1
        else:
            self.id = int(self.id)
            if self.id >= Line._id_counter:
                Line._id_counter = self.id + 1

        if self.name is None:
            self.name = f"Line {self.id}"
        else:
            self.name = str(self.name)

        if self.from_bus.network != self.to_bus.network:
            raise ValueError("Both buses must belong to the same network.")
        
        self.network = self.from_bus.network #Add network to line
        self.network.lines.append(self) #Add line to network

    @property
    def zb(self) -> float:
        """Impedância base (pu)"""
        return self.vb**2 / self.pb

    @property
    def resistance(self) -> float:
        """Resistência da linha (pu)"""
        return self.r / self.zb

    @property
    def reactance(self) -> float:
        """Reatância da linha (pu)"""
        return self.x / self.zb

    @property
    def shunt_admittance_half(self) -> float:
        """Admitância shunt (half) (pu)"""
        return self.b_half / self.zb

    @property
    def impedance(self) -> complex:
        """Impedância da linha (ohms)"""
        return complex(self.resistance, self.reactance)

    @property
    def admittance(self) -> complex:
        """Admitância da linha (S)"""
        return 1 / self.impedance

    @property
    def tap_phase_rad(self) -> float:
        """Fase de tap em radianos"""
        return np.deg2rad(self.tap_phase)

    def get_admittance_elements(self, bus_index: Dict[str, int]):
        """Gera os elementos de admitância baseados nos parâmetros da linha"""
        y = self.admittance
        b = self.shunt_admittance_half * 1j
        a = self.tap_ratio * np.exp(1j * self.tap_phase_rad)
        i = bus_index[self.from_bus.id]
        j = bus_index[self.to_bus.id]
        if self.tap_ratio != 1.0 or self.tap_phase != 0.0:
            Yff = y / (a * np.conj(a)) + b
            Yft = -y / np.conj(a)
            Ytf = -y / a
            Ytt = y + b
        else:
            Yff = y + b
            Yft = -y
            Ytf = -y
            Ytt = y + b
        return [((i, i), Yff), ((i, j), Yft), ((j, i), Ytf), ((j, j), Ytt)]

    def __repr__(self):
        return (f"Line(id={self.name}, Barra para:{self.from_bus.id}, Barra de:{self.to_bus.id}, r={self.resistance:.4f}, x={self.reactance:.4f})")

## Network

In [7]:
@dataclass
class Network:
    id: Optional[int] = None
    name: Optional[str] = None
    buses: List[Bus] = field(default_factory=list)
    lines: List[Line] = field(default_factory=list)
    loads: List[Load] = field(default_factory=list)
    generators: List[Generator] = field(default_factory=list)

    def y_bus(self) -> np.ndarray:
        """
        Returns the Y bus matrix of the network.
        """
        n = len(self.buses)
        bus_idx = {bus.id: i for i, bus in enumerate(self.buses)}
        ybus = np.zeros((n, n), dtype=complex)
        for line in self.lines:
            for (i, j), y in line.get_admittance_elements(bus_idx):
                ybus[i, j] += y
        return ybus
        
    def get_G(self):
        return self.y_bus().real
    
    def get_B(self):
        return self.y_bus().imag
    
    def __repr__(self):
        return f"Network(id={self.id}, name={self.name})"

In [8]:
net = Network()

In [9]:
net = Network()
# Criação das barras
buses = [
    Bus(net, id=1, bus_type='Slack'),
    Bus(net, id=2, bus_type='PV'), 
    Bus(net, id=3)
]
print(net.buses)

[Bus(id=1, type=Slack, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=2, type=PV, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=3, type=PQ, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0)]


In [10]:
net = Network()
# Criação das barras
buses = [
    Bus(net, id=1, bus_type='Slack'),
    Bus(net, id=2, bus_type='PV'), 
    Bus(net, id=3)
]
print(net.buses)
# Criação das linhas
lines = [
    Line(id=1, from_bus=buses[0], to_bus=buses[1], r=0.01938, x=0.05917, b_half=0.00264),
    Line(id=2, from_bus=buses[0], to_bus=buses[2], r=0.05403, x=0.22304, b_half=0.00264),
    Line(id=3, from_bus=buses[1], to_bus=buses[2], r=0.04699, x=0.19797, b_half=0.00219),
]
print(net.lines)

[Bus(id=1, type=Slack, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=2, type=PV, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=3, type=PQ, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0)]
[Line(id=Line 1, Barra para:1, Barra de:2, r=0.0194, x=0.0592), Line(id=Line 2, Barra para:1, Barra de:3, r=0.0540, x=0.2230), Line(id=Line 3, Barra para:2, Barra de:3, r=0.0470, x=0.1980)]


In [11]:
net = Network()
# Criação das barras
buses = [
    Bus(net, id=1, bus_type='Slack'),
    Bus(net, id=2, bus_type='PV'), 
    Bus(net, id=3)
]
print(net.buses)
# Criação das linhas
lines = [
    Line(id=1, from_bus=buses[0], to_bus=buses[1], r=0.01938, x=0.05917, b_half=0.00264),
    Line(id=2, from_bus=buses[0], to_bus=buses[2], r=0.05403, x=0.22304, b_half=0.00264),
    Line(id=3, from_bus=buses[1], to_bus=buses[2], r=0.04699, x=0.19797, b_half=0.00219),
]
print(net.lines)
y = net.y_bus()
Y = pd.DataFrame(y)
Y
#Criação de Geradores:
generators = [
    Generator(id=1, bus=buses[0]), #Gerador Vtheta
    Generator(id=2, bus=buses[1]), #Compensador Síncrono
]

print(net.generators)
loads = [
    Load(id=1, bus=buses[1], pb=1000, p_input=240, q_input=120),
    Load(id=2, bus=buses[2], pb=1000, p_input=240, q_input=120)
]

print(net.loads)


[Bus(id=1, type=Slack, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=2, type=PV, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=3, type=PQ, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0)]
[Line(id=Line 1, Barra para:1, Barra de:2, r=0.0194, x=0.0592), Line(id=Line 2, Barra para:1, Barra de:3, r=0.0540, x=0.2230), Line(id=Line 3, Barra para:2, Barra de:3, r=0.0470, x=0.1980)]
[Generator(id=1, bus=1, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None]), Generator(id=2, bus=2, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None])]
[Load(id=1, bus=2, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None]), Load(id=2, bus=3, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None])]


In [12]:
#Criação de Geradores:
generators = [
    Generator(id=1, bus=buses[0]), #Gerador Vtheta
    Generator(id=2, bus=buses[1]), #Compensador Síncrono
]

print(net.generators)

[Generator(id=1, bus=1, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None]), Generator(id=2, bus=2, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None]), Generator(id=1, bus=1, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None]), Generator(id=2, bus=2, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None])]


In [13]:
loads = [
    Load(id=1, bus=buses[1], pb=1000, p_input=240, q_input=120),
    Load(id=2, bus=buses[2], pb=1000, p_input=240, q_input=120)
]

print(net.loads)

[Load(id=1, bus=2, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None]), Load(id=2, bus=3, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None]), Load(id=1, bus=2, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None]), Load(id=2, bus=3, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None])]


## AC_PF

In [ ]:
class AC_PF:
    def __init__(self, network: Network, tol_P: float=1e-6, tol_Q: float=1e-6, max_iter: int=20):
        """
        Initializes the AC Power Flow class.
        """
        self.network = network # Network object
        self.tol_P = tol_P # P Tolerance
        self.tol_Q = tol_Q # Q Tolerance
        self.iter = 0 # Iteration counter
        self.max_iter = max_iter # Maximum number of iterations
        self.G = self.network.get_G() # Real part of YBUS
        self.B = self.network.get_B() # Imaginary part of YBUS

        #Organize bus types:
        self.pq_buses = [bus for bus in self.network.buses if bus.bus_type == 'PQ'] # PQ buses
        self.pv_buses = [bus for bus in self.network.buses if bus.bus_type == 'PV'] # PV buses
        self.slack_bus = [bus for bus in self.network.buses if bus.bus_type == 'Slack'] # Slack bus

        self.P_esp = [bus.p for bus in self.network.buses]
        self.Q_esp = [bus.q for bus in self.network.buses]

        #Initial V and theta
        #self.V_0 = [bus.v for bus in self.network.buses]
        #self.theta_0 = [bus.theta for bus in self.network.buses]
        self.V_0 = [1 for bus in self.network.buses]
        self.theta_0 = [0 for bus in self.network.buses]

        # Bus Map:
        self.bus_idx = {bus.id: i for i, bus in enumerate(self.network.buses)}

    def get_K_set(self):
        """
        Returns a dictionary where the keys are buses and the values are the sets of buses that are connected to the bus 
        including the bus itself.
        """
        K_set = {}

        #Iterate over all buses
        for bus in self.network.buses:
            connected_buses = {bus} #Starts the set with the bus itself
            #add all buses connected to the bus
            for line in self.network.lines:
                if line.from_bus == bus:
                    connected_buses.add(line.to_bus)
                elif line.to_bus == bus:
                    connected_buses.add(line.from_bus)
        
            #Ensure connected_buses is a set (so it will not have duplicates)
            connected_buses = set(connected_buses)

            #Add the set to the dictionary
            K_set[bus] = connected_buses

        return K_set

    def get_Omega_set(self):
        """
        Returns a dictionary where the keys are buses and the values are the sets of buses that are connected to the bus
        """

        omega_set = {}

        #Iterate over all buses
        for bus in self.network.buses:
            connected_buses = set() # Start with an empty set
            for line in self.network.lines:
                if line.from_bus == bus:
                    connected_buses.add(line.to_bus)
                elif line.to_bus == bus:
                    connected_buses.add(line.from_bus)
        
            #Ensure connected_buses is a set (so it will not have duplicates)
            connected_buses = set(connected_buses)

            #Add the set to the dictionary
            omega_set[bus] = connected_buses
        
        return omega_set
    
    # Method for power equations: It receives current V and theta for all buses and returns calculated P's and Q's.
    def pq_calc(self, V, theta): #Ta certo e ja conferi

        # Delta is a matrix that stores angle difference between all buses.
        delta = np.outer(theta, np.ones(len(self.network.buses))) - np.outer(np.ones(len(self.network.buses)), theta)
        
        # Calculates Real Power in a vetorial way
        P = np.dot(V, (V * (self.G * np.cos(delta) + self.B * np.sin(delta))).T)

        # Calculates Reactive Power in a vetorial way
        Q = np.dot(V, (V * (self.G * np.sin(delta) - self.B * np.cos(delta))).T)

        # If we want to return a single vector where P's and Q's are intercalated
        #Pq = np.empty(2*len(P))
        #PQ[0::2] = P
        #PQ[1::2] = Q

        return P, Q


    # Method to calculate Power Mismatches
    def mismatch(self, P, Q):

        dP = self.P_esp - P
        dQ = self.Q_esp - Q

        #Zera mismatch das variáveis fixadas:
        for bus in self.slack_bus:
            dP[self.bus_idx[bus.id]] = 0
            dQ[self.bus_idx[bus.id]] = 0

        for bus in self.pv_buses:
            dQ[self.bus_idx[bus.id]] = 0
            
        return dP, dQ
    
    def jacobian(self, V, theta):
        """
        Constructs the Jacobian matrix for the Newton-Raphson power flow method.

        The Jacobian is structured as:
            | H  N |
            | M  L |

        Where:
            H = ∂P/∂θ (theta partial derivatives of real power)
            N = ∂P/∂V (voltage magnitude partial derivatives of real power)
            M = ∂Q/∂θ (theta partial derivatives of reactive power)
            L = ∂Q/∂V (voltage magnitude partial derivatives of reactive power)

        Note:
            - Only PQ and PV buses are included in real power (P) equations.
            - Only PQ buses are included in reactive power (Q) equations.
            - However, we construct a full Jacobian (2N x 2N) including all buses and use a large number (big-M)
              on diagonal elements corresponding to fixed variables (Slack θ and PV/Slack V) to enforce their values 
              during updates.

        Returns:
            J (ndarray): The complete Jacobian matrix (2 * n_bus x 2 * n_bus), ready for use in Newton-Raphson updates.
        """

        n_bus = len(self.network.buses)
        big_number = 1e10

        # Create voltage angle difference matrix Δθ_ij = θ_i - θ_j
        delta = np.outer(theta, np.ones(n_bus)) - np.outer(np.ones(n_bus), theta)

        # Initialize full matrices H, N, M, L of size n_bus x n_bus
        H = np.zeros((n_bus, n_bus))  # ∂P/∂θ
        N = np.zeros((n_bus, n_bus))  # ∂P/∂V
        M = np.zeros((n_bus, n_bus))  # ∂Q/∂θ
        L = np.zeros((n_bus, n_bus))  # ∂Q/∂V

        for i in range(n_bus):
            for j in range(n_bus):
                if i == j:
                    # Diagonal elements (derivatives with respect to same bus)

                    # ∂P_i / ∂θ_i (H matrix diagonal)
                    H[i, i] = -np.sum(
                        V[i] * V * (self.G[i, :] * np.sin(delta[i, :]) - self.B[i, :] * np.cos(delta[i, :]))
                    )

                    # ∂P_i / ∂V_i (N matrix diagonal)
                    N[i, i] = V[i] * self.G[i, i] + np.sum(
                        V * (self.G[i, :] * np.cos(delta[i, :]) + self.B[i, :] * np.sin(delta[i, :]))
                    )

                    # ∂Q_i / ∂θ_i (M matrix diagonal)
                    M[i, i] = -np.sum(
                        V[i] * V * (self.G[i, :] * np.cos(delta[i, :]) + self.B[i, :] * np.sin(delta[i, :]))
                    )

                    # ∂Q_i / ∂V_i (L matrix diagonal)
                    L[i, i] = V[i] * self.B[i, i] + np.sum(
                        V * (self.G[i, :] * np.sin(delta[i, :]) - self.B[i, :] * np.cos(delta[i, :]))
                    )
                else:
                    # Off-diagonal elements (derivatives with respect to other buses)

                    # ∂P_i / ∂θ_j (H matrix off-diagonal)
                    H[i, j] = V[i] * V[j] * (
                        self.G[i, j] * np.sin(delta[i, j]) - self.B[i, j] * np.cos(delta[i, j])
                    )

                    # ∂P_i / ∂V_j (N matrix off-diagonal)
                    N[i, j] = V[i] * (
                        self.G[i, j] * np.cos(delta[i, j]) + self.B[i, j] * np.sin(delta[i, j])
                    )

                    # ∂Q_i / ∂θ_j (M matrix off-diagonal)
                    M[i, j] = -V[i] * V[j] * (
                        self.G[i, j] * np.cos(delta[i, j]) + self.B[i, j] * np.sin(delta[i, j])
                    )

                    # ∂Q_i / ∂V_j (L matrix off-diagonal)
                    L[i, j] = V[i] * (
                        self.G[i, j] * np.sin(delta[i, j]) - self.B[i, j] * np.cos(delta[i, j])
                    )

        # Concatenate final Jacobian
        top = np.hstack([H, N])
        bottom = np.hstack([M, L])
        J = np.vstack([top, bottom])

        # Fix variables for slack and PV buses by applying large constants to diagonal
        theta_fixed_idx = [self.bus_idx[bus.id] for bus in self.slack_bus]
        v_fixed_idx = [self.bus_idx[bus.id] for bus in self.pv_buses + self.slack_bus]

        for i in theta_fixed_idx:
            J[i,i] = big_number
        
        for i in v_fixed_idx:
            J[i + n_bus, i + n_bus] = big_number

        return J
    

    def solve(self):
        """
        Solves the power flow problem using the Newton-Raphson method.
        """
        # Initialization
        V = np.array(self.V_0, dtype=float)
        print(V)
        theta = np.array(self.theta_0, dtype=float)
        print(theta)

        for self.iter in range(self.max_iter):
            P, Q = self.pq_calc(V, theta)
            print(f"Potencia P calculada: {P}")
            print(f"Potencia Q calculada: {Q}")
            dP, dQ = self.mismatch(P, Q)

            dPdQ = np.concatenate([dP, dQ])
            print(f"Mismatch: {dPdQ}")
            
            max_mismatch = max(np.linalg.norm(dP, np.inf), np.linalg.norm(dQ, np.inf))
            if max_mismatch < max(self.tol_P, self.tol_Q):
                print("Power flow converged in {} iterations.".format(self.iter + 1))
                break
            
            # Calculate the Jacobian
            J = self.jacobian(V, theta)

            # Solve the linear system
            dx = np.linalg.solve(J, dPdQ)

            # Update θ and V
            theta += dx[:len(V)]
            V += dx[len(V):]
        
        else:
            print("Power flow did not converge after max_iter iterations.")
        
        # Update the bus voltages and angles
        for i, bus in enumerate(self.network.buses):
            bus.v = V[i]
            bus.theta = theta[i]

        print(f"Power flow converged after {self.iter} iterations.")

        return None





In [87]:
class AC_PF:
    def __init__(self, network: Network, tol_P: float=1e-6, tol_Q: float=1e-6, max_iter: int=20):
        """
        Initializes the AC Power Flow class.
        """
        self.network = network # Network object
        self.tol_P = tol_P # P Tolerance
        self.tol_Q = tol_Q # Q Tolerance
        self.iter = 0 # Iteration counter
        self.max_iter = max_iter # Maximum number of iterations
        self.G = self.network.get_G() # Real part of YBUS
        self.B = self.network.get_B() # Imaginary part of YBUS

        #Organize bus types:
        self.pq_buses = [bus for bus in self.network.buses if bus.bus_type == 'PQ'] # PQ buses
        self.pv_buses = [bus for bus in self.network.buses if bus.bus_type == 'PV'] # PV buses
        self.slack_bus = [bus for bus in self.network.buses if bus.bus_type == 'Slack'] # Slack bus

        self.P_esp = np.array([bus.p for bus in self.network.buses])
        self.Q_esp = np.array([bus.q for bus in self.network.buses])

        #Initial V and theta
        self.V_0 = np.array([bus.v for bus in self.network.buses])
        self.theta_0 = np.array([bus.theta for bus in self.network.buses])
        #self.V_0 = np.array([1 for bus in self.network.buses])
        #self.theta_0 = np.array([0 for bus in self.network.buses])

        # Bus Map:
        self.bus_idx = {bus.id: i for i, bus in enumerate(self.network.buses)}

    def get_K_set(self):
        """
        Returns a dictionary where the keys are buses and the values are the sets of buses that are connected to the bus 
        including the bus itself.
        """
        K_set = {}

        #Iterate over all buses
        for bus in self.network.buses:
            connected_buses = {bus} #Starts the set with the bus itself
            #add all buses connected to the bus
            for line in self.network.lines:
                if line.from_bus == bus:
                    connected_buses.add(line.to_bus)
                elif line.to_bus == bus:
                    connected_buses.add(line.from_bus)
        
            #Ensure connected_buses is a set (so it will not have duplicates)
            connected_buses = set(connected_buses)

            #Add the set to the dictionary
            K_set[bus] = connected_buses

        return K_set

    def get_Omega_set(self):
        """
        Returns a dictionary where the keys are buses and the values are the sets of buses that are connected to the bus
        """

        omega_set = {}

        #Iterate over all buses
        for bus in self.network.buses:
            connected_buses = set() # Start with an empty set
            for line in self.network.lines:
                if line.from_bus == bus:
                    connected_buses.add(line.to_bus)
                elif line.to_bus == bus:
                    connected_buses.add(line.from_bus)
        
            #Ensure connected_buses is a set (so it will not have duplicates)
            connected_buses = set(connected_buses)

            #Add the set to the dictionary
            omega_set[bus] = connected_buses
        
        return omega_set
    
    # Method for power equations: It receives current V and theta for all buses and returns calculated P's and Q's.
    def pq_calc(self, V, theta):

        # Delta is a matrix that stores angle difference between all buses.
        delta = np.outer(theta, np.ones(len(self.network.buses))) - np.outer(np.ones(len(self.network.buses)), theta)
        
        # Calculates Real Power in a vetorial way
        P = np.dot(V, (V * (self.G * np.cos(delta) + self.B * np.sin(delta))).T)

        # Calculates Reactive Power in a vetorial way
        Q = np.dot(V, (V * (self.G * np.sin(delta) - self.B * np.cos(delta))).T)

        # If we want to return a single vector where P's and Q's are intercalated
        #Pq = np.empty(2*len(P))
        #PQ[0::2] = P
        #PQ[1::2] = Q

        return P, Q
    

    # Method to calculate Power Mismatches
    def mismatch(self, P, Q):

        dP = self.P_esp - P
        dQ = self.Q_esp - Q

        #Zera mismatch das variáveis fixadas:
        for bus in self.slack_bus:
            dP[self.bus_idx[bus.id]] = 0
            dQ[self.bus_idx[bus.id]] = 0

        for bus in self.pv_buses:
            dQ[self.bus_idx[bus.id]] = 0
            
        return dP, dQ
    
    def jacobian(self, V, theta):
        """
        Método tradicional para montar a matriz Jacobiana do fluxo de carga via Newton-Raphson.

        As matrizes são:
            H = ∂P/∂θ
            N = ∂P/∂V
            M = ∂Q/∂θ
            L = ∂Q/∂V

        O Jacobiano final é:
            | H  N |
            | M  L |

        Elementos fixos (slack e PV) são tratados com um valor grande (big-M) para forçar suas condições.
        """
        n_bus = len(self.network.buses)
        big_number = 1e5

        # Inicializa as matrizes H, N, M, L
        H = np.zeros((n_bus, n_bus))
        N = np.zeros((n_bus, n_bus))
        M = np.zeros((n_bus, n_bus))
        L = np.zeros((n_bus, n_bus))

        for i in range(n_bus):
            for j in range(n_bus):
                theta_diff = theta[i] - theta[j]
                G_ij = self.G[i][j]
                B_ij = self.B[i][j]

                if i == j:
                    # Elementos diagonais
                    h = 0
                    n = G_ij * V[i]
                    m = 0
                    l = -B_ij * V[i]

                    for k in range(n_bus):
                        if k == i:
                            continue
                        G_ik = self.G[i][k]
                        B_ik = self.B[i][k]
                        theta_diff_ik = theta[i] - theta[k]

                        h += V[i] * V[k] * (G_ik * np.sin(theta_diff_ik) - B_ik * np.cos(theta_diff_ik))
                        n += V[k] * (G_ik * np.cos(theta_diff_ik) + B_ik * np.sin(theta_diff_ik))
                        m += V[i] * V[k] * (G_ik * np.cos(theta_diff_ik) + B_ik * np.sin(theta_diff_ik))
                        l += V[k] * (G_ik * np.sin(theta_diff_ik) - B_ik * np.cos(theta_diff_ik))

                    H[i][i] = -h
                    N[i][i] = n
                    M[i][i] = -m
                    L[i][i] = l
                else:
                    # Elementos fora da diagonal
                    H[i][j] = V[i] * V[j] * (G_ij * np.sin(theta_diff) - B_ij * np.cos(theta_diff))
                    N[i][j] = V[i] * (G_ij * np.cos(theta_diff) + B_ij * np.sin(theta_diff))
                    M[i][j] = -V[i] * V[j] * (G_ij * np.cos(theta_diff) + B_ij * np.sin(theta_diff))
                    L[i][j] = V[i] * (G_ij * np.sin(theta_diff) - B_ij * np.cos(theta_diff))

        # Monta Jacobiano completo
        top = np.hstack([H, N])
        bottom = np.hstack([M, L])
        J = np.vstack([top, bottom])

        # Corrige variáveis fixas
        theta_fixed_idx = [self.bus_idx[bus.id] for bus in self.slack_bus]
        v_fixed_idx = [self.bus_idx[bus.id] for bus in self.pv_buses + self.slack_bus]

        for i in theta_fixed_idx:
            for j in range(2 * n_bus):
                J[i][j] = 0
            J[i][i] = big_number

        for i in v_fixed_idx:
            for j in range(2 * n_bus):
                J[i + n_bus][j] = 0
            J[i + n_bus][i + n_bus] = big_number

        return J

    
    def jacobian1(self, V, theta):
        """
        Constructs the Jacobian matrix for the Newton-Raphson power flow method.

        The Jacobian is structured as:
            | H  N |
            | M  L |

        Where:
            H = ∂P/∂θ (theta partial derivatives of real power)
            N = ∂P/∂V (voltage magnitude partial derivatives of real power)
            M = ∂Q/∂θ (theta partial derivatives of reactive power)
            L = ∂Q/∂V (voltage magnitude partial derivatives of reactive power)

        Note:
            - Only PQ and PV buses are included in real power (P) equations.
            - Only PQ buses are included in reactive power (Q) equations.
            - However, we construct a full Jacobian (2N x 2N) including all buses and use a large number (big-M)
              on diagonal elements corresponding to fixed variables (Slack θ and PV/Slack V) to enforce their values 
              during updates.

        Returns:
            J (ndarray): The complete Jacobian matrix (2 * n_bus x 2 * n_bus), ready for use in Newton-Raphson updates.
        """

        n_bus = len(self.network.buses)
        big_number = 1e10

        # Create voltage angle difference matrix Δθ_ij = θ_i - θ_j
        delta = np.outer(theta, np.ones(n_bus)) - np.outer(np.ones(n_bus), theta)

        # Initialize full matrices H, N, M, L of size n_bus x n_bus
        H = np.zeros((n_bus, n_bus))  # ∂P/∂θ
        N = np.zeros((n_bus, n_bus))  # ∂P/∂V
        M = np.zeros((n_bus, n_bus))  # ∂Q/∂θ
        L = np.zeros((n_bus, n_bus))  # ∂Q/∂V

        for i in range(n_bus):
            for j in range(n_bus):
                if i == j:
                    # Diagonal elements (derivatives with respect to same bus)

                    # ∂P_i / ∂θ_i (H matrix diagonal)
                    H[i, i] = -np.sum(
                        V[i] * V * (self.G[i, :] * np.sin(delta[i, :]) - self.B[i, :] * np.cos(delta[i, :]))
                    )

                    # ∂P_i / ∂V_i (N matrix diagonal)
                    N[i, i] = V[i] * self.G[i, i] + np.sum(
                        V * (self.G[i, :] * np.cos(delta[i, :]) + self.B[i, :] * np.sin(delta[i, :]))
                    )

                    # ∂Q_i / ∂θ_i (M matrix diagonal)
                    M[i, i] = -np.sum(
                        V[i] * V * (self.G[i, :] * np.cos(delta[i, :]) + self.B[i, :] * np.sin(delta[i, :]))
                    )

                    # ∂Q_i / ∂V_i (L matrix diagonal)
                    L[i, i] = V[i] * self.B[i, i] + np.sum(
                        V * (self.G[i, :] * np.sin(delta[i, :]) - self.B[i, :] * np.cos(delta[i, :]))
                    )
                else:
                    # Off-diagonal elements (derivatives with respect to other buses)

                    # ∂P_i / ∂θ_j (H matrix off-diagonal)
                    H[i, j] = V[i] * V[j] * (
                        self.G[i, j] * np.sin(delta[i, j]) - self.B[i, j] * np.cos(delta[i, j])
                    )

                    # ∂P_i / ∂V_j (N matrix off-diagonal)
                    N[i, j] = V[i] * (
                        self.G[i, j] * np.cos(delta[i, j]) + self.B[i, j] * np.sin(delta[i, j])
                    )

                    # ∂Q_i / ∂θ_j (M matrix off-diagonal)
                    M[i, j] = -V[i] * V[j] * (
                        self.G[i, j] * np.cos(delta[i, j]) + self.B[i, j] * np.sin(delta[i, j])
                    )

                    # ∂Q_i / ∂V_j (L matrix off-diagonal)
                    L[i, j] = V[i] * (
                        self.G[i, j] * np.sin(delta[i, j]) - self.B[i, j] * np.cos(delta[i, j])
                    )

        # Concatenate final Jacobian
        top = np.hstack([H, N])
        bottom = np.hstack([M, L])
        J = np.vstack([top, bottom])

        # Fix variables for slack and PV buses by applying large constants to diagonal
        theta_fixed_idx = [self.bus_idx[bus.id] for bus in self.slack_bus]
        v_fixed_idx = [self.bus_idx[bus.id] for bus in self.pv_buses + self.slack_bus]

        for i in theta_fixed_idx:
            J[i, :] = 0
            J[i,i] = big_number
        
        for i in v_fixed_idx:
            J[i + n_bus, :] = 0
            J[i + n_bus, i + n_bus] = big_number

        return J
    

    def solve(self):
        """
        Solves the power flow problem using the Newton-Raphson method.
        """
        # Initialization
        V = np.array(self.V_0, dtype=float)
        print(V)
        theta = np.array(self.theta_0, dtype=float)
        print(theta)

        for self.iter in range(self.max_iter):
            P, Q = self.pq_calc(V, theta)
            print(f"Potencia P calculada: {P}")
            print(f"Potencia Q calculada: {Q}")
            
            dP, dQ = self.mismatch(P, Q)

            dPdQ = np.concatenate([dP, dQ])
            print(f"Mismatch: {dPdQ}")
            
            max_mismatch = max(np.linalg.norm(dP, np.inf), np.linalg.norm(dQ, np.inf))
            if max_mismatch < max(self.tol_P, self.tol_Q):
                print("Power flow converged in {} iterations.".format(self.iter + 1))
                break
            
            # Calculate the Jacobian
            J = self.jacobian(V, theta)
            print(f"Jacobiano: {J}")

            # Solve the linear system
            dx = np.linalg.solve((J), dPdQ)

            # Update θ and V
            theta += dx[:len(V)]
            V += dx[len(V):]
        
        else:
            print("Power flow did not converge after max_iter iterations.")
        
        # Update the bus voltages and angles
        for i, bus in enumerate(self.network.buses):
            bus.v = V[i]
            bus.theta = theta[i]

        print(f"Power flow converged after {self.iter} iterations.")

        return None



In [88]:
net = Network()
# Criação das barras
buses = [
    Bus(net, id=1, bus_type='Slack'),
    Bus(net, id=2, bus_type='PV'), 
    Bus(net, id=3)
]
print(net.buses)
# Criação das linhas
lines = [
    Line(id=1, from_bus=buses[0], to_bus=buses[1], r=0.01938, x=0.05917, b_half=0.00264),
    Line(id=2, from_bus=buses[0], to_bus=buses[2], r=0.05403, x=0.22304, b_half=0.00264),
    Line(id=3, from_bus=buses[1], to_bus=buses[2], r=0.04699, x=0.19797, b_half=0.00219),
]
print(net.lines)
y = net.y_bus()
Y = pd.DataFrame(y)
Y
#Criação de Geradores:
generators = [
    Generator(id=1, bus=buses[0]), #Gerador Vtheta
    Generator(id=2, bus=buses[1]), #Compensador Síncrono
]

print(net.generators)
loads = [
    Load(id=1, bus=buses[1], pb=1000, p_input=240, q_input=120),
    Load(id=2, bus=buses[2], pb=1000, p_input=240, q_input=120)
]

print(net.loads)


[Bus(id=1, type=Slack, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=2, type=PV, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0), Bus(id=3, type=PQ, v=1.000 pu, theta=0.000 rad, p=0.000 pu, q=0.000 pu, gen=0, load=0)]
[Line(id=Line 1, Barra para:1, Barra de:2, r=0.0194, x=0.0592), Line(id=Line 2, Barra para:1, Barra de:3, r=0.0540, x=0.2230), Line(id=Line 3, Barra para:2, Barra de:3, r=0.0470, x=0.1980)]
[Generator(id=1, bus=1, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None]), Generator(id=2, bus=2, p=0.000, q=0.000, p_range=[0.000,inf], q_range=[None,None])]
[Load(id=1, bus=2, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None]), Load(id=2, bus=3, p=0.240, q=0.120, p_range=[0.000,inf], q_range=[None,None])]


In [89]:
solver = AC_PF(net, max_iter = 20)


In [92]:
print(solver.P_esp)
print(solver.Q_esp)
print(solver.V_0)
print(solver.theta_0)

[ 0.   -0.24 -0.24]
[ 0.   -0.12 -0.12]
[1. 1. 1.]
[0. 0. 0.]


In [90]:
solver.solve()

[1. 1. 1.]
[0. 0. 0.]
Potencia P calculada: [ 2.22044605e-16 -2.22044605e-16  0.00000000e+00]
Potencia Q calculada: [-0.00528 -0.00483 -0.00483]
Mismatch: [ 0.      -0.24    -0.24     0.       0.      -0.11517]
Jacobiano: [[ 1.00000000e+05  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.52630865e+01  2.00449497e+01 -4.78186315e+00 -4.99913160e+00
  -2.22044605e-16 -1.13501919e+00]
 [-4.23498368e+00 -4.78186315e+00  9.01684683e+00 -1.02589745e+00
  -1.13501919e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+05
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+05  0.00000000e+00]
 [ 1.02589745e+00  1.13501919e+00  2.16091665e+00 -4.23498368e+00
  -4.78186315e+00 -4.83000000e-03]]
Potencia P calculada: [ 0.67341004 -0.17505707 -0.52512563]
Potencia Q calculada: [ 0.40059949  0.74631412 -1.1363811 ]
Mismatch: [ 0.         -0.06494293  0.28512563  0. 

In [86]:
solver.solve()

[1. 1. 1.]
[0. 0. 0.]
Potencia P calculada: [np.float64(2.220446049250313e-16), np.float64(-2.220446049250313e-16), np.float64(0.0)]
Potencia Q calculada: [np.float64(-0.005280000000001728), np.float64(-0.0048299999999974474), np.float64(-0.004830000000000112)]
Potencia P_ calculada: [ 2.22044605e-16 -2.22044605e-16  0.00000000e+00]
Potencia Q_ calculada: [-0.00528 -0.00483 -0.00483]
Mismatch: [ 0.      -0.24    -0.24     0.       0.      -0.11517]
Jacobiano: [[ 1.00000000e+05  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.52630865e+01  2.00449497e+01 -4.78186315e+00 -4.99913160e+00
  -2.22044605e-16 -1.13501919e+00]
 [-4.23498368e+00 -4.78186315e+00  9.01684683e+00 -1.02589745e+00
  -1.13501919e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+05
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+05  0.00000000e+00]
 [ 1.02589745e+00  1.13501919e+

In [29]:
print(buses[0].v)


1.0
